## Régression

#### Import des librairies

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from itertools import combinations
import scipy.stats as stats

#### Read csv

In [4]:
try:
  df_merged = pd.read_csv("C:/Users/Ihnhn/Desktop/M2 SISE/Python/merged_69.csv", sep=";")
except FileNotFoundError:
  print("Le chemin n'est pas bon")

C:\Users\Ihnhn\AppData\Local\Temp\ipykernel_7456\1118220225.py:2: DtypeWarning: Columns (33,35,38,51,57,69,72,99,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv("C:/Users/Ihnhn/Desktop/M2 SISE/Python/merged_69.csv", sep=";")


#### Analyse dataframe

In [5]:
df_merged.shape
#doit renvoyer 287165,131

(287165, 131)

In [6]:
# Afficher la liste complète des noms de colonnes pour identifier les cibles "étiquettes DPE" et "consommation énergétique"
columns = df_merged.columns.tolist()
columns

['Ubat_W/m²_K',
 "Complément_d'adresse_bâtiment",
 'Méthode_application_DPE',
 'N°_département_(BAN)',
 'Qualité_isolation_menuiseries',
 'Nombre_appartement',
 'Emission_GES_5_usages_énergie_n°2',
 'Coordonnée_cartographique_X_(BAN)',
 'Date_fin_validité_DPE',
 'Protection_solaire_exterieure_(0/1)',
 'Coût_total_5_usages',
 'Conso_ECS_dépensier_é_primaire',
 'Catégorie_ENR',
 'Présence_brasseur_air_(0/1)',
 'Date_établissement_DPE',
 'Score_BAN',
 'Type_énergie_n°2',
 'Conso_auxiliaires_é_primaire',
 'N°_étage_appartement',
 'N°_région_(BAN)',
 'Deperditions_planchers_hauts',
 'Adresse_brute',
 'Emission_GES_refroidissement_dépensier',
 'Version_DPE',
 'Deperditions_planchers_bas',
 'Surface_tertiaire_immeuble',
 'Coût_chauffage',
 '_i',
 'Qualité_isolation_plancher_bas',
 'Déperditions_renouvellement_air',
 'Surface_habitable_logement',
 'Type_énergie_n°1',
 'Conso_chauffage_é_finale',
 'Besoin_refroidissement',
 'Emission_GES_éclairage',
 'Code_INSEE_(BAN)',
 'Conso_refroidissement_

#### Pre processing

In [7]:
df = df_merged

In [8]:
#Fonction pour identifier le % de valeurs manquantes dans chaque colonne
#Renvoie seulement les colonnes contenant des valeurs manquantes + les tri
def percent_missing(df):
  percent_nan = 100 * df.isnull().sum() / len(df)
  percent_nan = percent_nan[percent_nan > 0].sort_values()

  return percent_nan

percent_nan = percent_missing(df)
percent_nan

Ubat_W/m²_K                            0.000348
Conso_auxiliaires_é_finale             0.000348
Conso_chauffage_é_primaire             0.002089
Emission_GES_chauffage                 0.002089
Conso_chauffage_dépensier_é_finale     0.002089
                                        ...    
Position_logement_dans_immeuble       98.864416
Typologie_logement                    98.864416
Cage_d'escalier                       99.464768
Surface_tertiaire_immeuble            99.531280
Invariant_fiscal_logement             99.596748
Length: 88, dtype: float64

In [9]:
#suppression des colonnes qui ont + de > 30% valeurs manquantes
df = df.drop(percent_nan[percent_nan > 30].index, axis=1)

In [10]:
#Vérification suppression des valeurs manquantes
percent_nan = percent_missing(df)
percent_nan

Ubat_W/m²_K                              0.000348
Conso_auxiliaires_é_finale               0.000348
Conso_chauffage_é_primaire               0.002089
Conso_chauffage_é_finale                 0.002089
Emission_GES_chauffage                   0.002089
Conso_chauffage_dépensier_é_primaire     0.002089
Conso_chauffage_dépensier_é_finale       0.002089
Emission_GES_chauffage_dépensier         0.002089
Coût_éclairage                           0.002438
Conso_5_usages_é_finale                  0.002786
Emission_GES_5_usages_par_m²             0.002786
Emission_GES_5_usages                    0.002786
Conso_5_usages_é_finale_énergie_n°1      0.002786
Coût_chauffage_énergie_n°1               0.002786
Coût_auxiliaires                         0.002786
Conso_5_usages_é_primaire                0.002786
Coût_refroidissement                     0.002786
Emission_GES_5_usages_énergie_n°1        0.002786
Conso_5_usages_par_m²_é_primaire         0.002786
Coût_ECS_énergie_n°1                     0.002786


In [13]:
#supprimer les lignes, dans les colonnes qui ont moins de 9% de valeurs manquantes
df = df.dropna(subset=percent_nan[percent_nan < 9].index, axis=0)

In [14]:
#Vérification suppression
percent_nan = percent_missing(df)
percent_nan

Type_installation_chauffage         9.713755
Type_installation_ECS_(général)     9.713755
Nombre_appartement                 10.340823
Complément_d'adresse_logement      17.877785
dtype: float64

##### Imputation des valeurs manquantes

In [15]:
# Sélectionner uniquement les colonnes quantitatives (numériques)
quant_cols = df.select_dtypes(include=[np.number]).columns

# Identifier les colonnes quantitatives avec des valeurs manquantes
cols_with_missing = percent_nan[percent_nan > 0].index.intersection(quant_cols)

# Imputation par la médiane pour les colonnes quantitatives avec des valeurs manquantes
for col in cols_with_missing:
    df[col] = df[col].fillna(df[col].median())

#Vérification des données manquantes
print(df[quant_cols].isnull().sum())


Ubat_W/m²_K                          0
N°_département_(BAN)                 0
Nombre_appartement                   0
Coordonnée_cartographique_X_(BAN)    0
Coût_total_5_usages                  0
                                    ..
Coût_auxiliaires                     0
Coût_refroidissement_dépensier       0
Emission_GES_ECS_dépensier           0
Conso_5_usages_par_m²_é_primaire     0
Besoin_chauffage                     0
Length: 69, dtype: int64


In [16]:
#Imputation des Données Manquantes sur variable qualitative

# Sélectionner toutes les colonnes non numériques (qualitatives)
categorical_cols = df.select_dtypes(exclude=[np.number]).columns

# Appliquer l'imputation par la valeur la plus fréquente (mode) pour chaque colonne catégorielle
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

#Vérification des données manquantes
df[categorical_cols].isnull().sum()

Méthode_application_DPE              0
Qualité_isolation_menuiseries        0
Date_fin_validité_DPE                0
Date_établissement_DPE               0
Adresse_brute                        0
Qualité_isolation_plancher_bas       0
Type_énergie_n°1                     0
Besoin_refroidissement               0
Code_INSEE_(BAN)                     0
Classe_altitude                      0
Adresse_(BAN)                        0
Date_visite_diagnostiqueur           0
_geopoint                            0
Qualité_isolation_enveloppe          0
Identifiant__BAN                     0
Statut_géocodage                     0
Besoin_ECS                           0
Type_installation_chauffage          0
_id                                  0
Type_bâtiment                        0
Date_réception_DPE                   0
N°DPE                                0
Type_énergie_principale_chauffage    0
Type_installation_ECS_(général)      0
Classe_inertie_bâtiment              0
Etiquette_GES            

In [17]:
col = df.columns.tolist()
col
# Sélectionner toutes les colonnes sauf celles contenant "Coût" ou "Conso" ou "GES" qui sont liées a la variable target pour éviter Data Leakage
les_variables_explicatives = [col for col in df.columns if not ("Coût" in col or "Conso" in col or "GES" in col or "coût" in col)]
#les_variables_explicatives

In [18]:
# Filtrer les colonnes quantitatives parmi les variables explicatives
quantitative_variables = [col for col in les_variables_explicatives if np.issubdtype(df[col].dtype, np.number)]
# Afficher les variables quantitatives sélectionnées
quantitative_variables

['Ubat_W/m²_K',
 'N°_département_(BAN)',
 'Nombre_appartement',
 'Coordonnée_cartographique_X_(BAN)',
 'Score_BAN',
 'N°_étage_appartement',
 'N°_région_(BAN)',
 'Deperditions_planchers_hauts',
 'Version_DPE',
 'Deperditions_planchers_bas',
 '_i',
 'Déperditions_renouvellement_air',
 'Surface_habitable_logement',
 '_score',
 'Nombre_niveau_logement',
 'Déperditions_ponts_thermiques',
 'Coordonnée_cartographique_Y_(BAN)',
 'Deperditions_enveloppe',
 'Hauteur_sous-plafond',
 'Déperditions_portes',
 'Code_postal_(BAN)',
 '_rand',
 'Déperditions_murs',
 'Deperditions_baies_vitrées',
 'Code_postal_(brut)',
 'Besoin_chauffage']

In [19]:
# Supprimer les colonnes constantes (qui ne donneront donc aucune corrélation)
quantitative_variables = [col for col in quantitative_variables if df[col].nunique() > 1]
# Calculer la corrélation entre chaque variable quantitative et 'Conso_5_usages_é_finale'
correlations = df[quantitative_variables].corrwith(df['Conso_5_usages_é_finale'])
# Trier les corrélations dans l'ordre décroissant
correlations_sorted = correlations.sort_values(ascending=False)
# Filtrer pour garder uniquement les corrélations supérieures à 0.2
correlations_filtered = correlations_sorted[correlations_sorted > 0.2]
# Afficher les corrélations filtrées
print(correlations_filtered)


Surface_habitable_logement    0.664904
Ubat_W/m²_K                   0.324088
dtype: float64


In [20]:
df['Conso_5_usages_é_finale'].describe()
#Il faut traiter les outliers de df1['Conso_5_usages_é_finale'] pour éviter que le modèle apprenne mal

count    189453.000000
mean      10399.339109
std        9198.048501
min         556.500000
25%        5015.400000
50%        8032.000000
75%       13158.900000
max      429196.900000
Name: Conso_5_usages_é_finale, dtype: float64

In [23]:
# Calculate Q1 and Q3
Q1 = df['Conso_5_usages_é_finale'].quantile(0.25)
Q3 = df['Conso_5_usages_é_finale'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Calculate the upper limit for outliers
limitSup = Q3 + (IQR * 1.5)
print(limitSup) #le seuil qui considère valeurs aberrantes niv sup

# Compter le nombre de valeurs supérieures à limitSup (outliers)
count_above_outliers = (df['Conso_5_usages_é_finale'] > limitSup).sum()

# Afficher le résultat
print("Nombre de valeurs supérieures à",limitSup,":", count_above_outliers)

# Calculer le pourcentage et arrondir à deux chiffres après la virgule
percentage_above_20000 = round((count_above_outliers / len(df['Conso_5_usages_é_finale'])) * 100, 2)

# Afficher le résultat
print("Pourcentage de valeurs supérieures à", limitSup, ":", percentage_above_20000, "%")


25374.15
Nombre de valeurs supérieures à 25374.15 : 8784
Pourcentage de valeurs supérieures à 25374.15 : 4.64 %


In [24]:
df = df[df['Conso_5_usages_é_finale'] <= limitSup] #suppression des outliers qui représentent 5% de la colonne


#### Sélection des variables explicatives

In [25]:
#Creation d'un autre dataframe pour test ANOVA (pour que ne pas encoder les données du dataframe)
#Car l encodage sera fait dans le pipeline ce qui permettra de reproduire pour les nouvelles données + simplement
df1 = df

In [26]:
#faire test stat pour évaluer les var qualitatives significatives avec variable cible

quantitative_var = 'Conso_5_usages_é_finale'
# Filtrer les variables qualitatives
qualitative_vars = df1.select_dtypes(include=['object', 'category']).columns
# Dictionnaire pour stocker les résultats
anova_results = {}

# Parcourir les variables qualitatives
for qual_var in qualitative_vars:
    # Groupement des données en une seule opération
    grouped_data = df1.groupby(qual_var)[quantitative_var].apply(list)

    # Filtrer les groupes avec au moins deux valeurs
    filtered_groups = [group for group in grouped_data if len(group) > 1]

    # Effectuer le test ANOVA si le nombre de groupes est supérieur à 1
    if len(filtered_groups) > 1:
        f_stat, p_value = stats.f_oneway(*filtered_groups)
        anova_results[qual_var] = {'F_statistic': f_stat, 'p_value': p_value}


In [27]:
# Trier les résultats par ordre décroissant de la statistique F
anova_results_sorted = dict(sorted(anova_results.items(), key=lambda item: item[1]['F_statistic'], reverse=True))
# Afficher les résultats triés
for var, result in anova_results_sorted.items():
    print(f"Variable qualitative : {var}")
    print(f"  Statistique F : {result['F_statistic']:.2f}, Valeur p : {result['p_value']:.4e}\n")

Variable qualitative : Etiquette_GES
  Statistique F : 20371.29, Valeur p : 0.0000e+00

Variable qualitative : Qualité_isolation_murs
  Statistique F : 12232.95, Valeur p : 0.0000e+00

Variable qualitative : Type_bâtiment
  Statistique F : 11700.46, Valeur p : 0.0000e+00

Variable qualitative : Qualité_isolation_enveloppe
  Statistique F : 7973.39, Valeur p : 0.0000e+00

Variable qualitative : Méthode_application_DPE
  Statistique F : 5868.83, Valeur p : 0.0000e+00

Variable qualitative : Etiquette_DPE
  Statistique F : 5307.98, Valeur p : 0.0000e+00

Variable qualitative : Type_installation_chauffage
  Statistique F : 4936.22, Valeur p : 0.0000e+00

Variable qualitative : Type_énergie_principale_chauffage
  Statistique F : 4399.18, Valeur p : 0.0000e+00

Variable qualitative : Qualité_isolation_plancher_bas
  Statistique F : 4227.10, Valeur p : 0.0000e+00

Variable qualitative : Type_énergie_n°1
  Statistique F : 3880.23, Valeur p : 0.0000e+00

Variable qualitative : Qualité_isolation

In [29]:
# 1. Filtrer les variables quantitatives avec une corrélation supérieure à 0,2 qui sont stockées dans correlations_filtered
#correlations_filtered = correlations_sorted[correlations_sorted > 0.2]
quantitative_vars = correlations_filtered.index.tolist()  # Liste des variables quantitatives sélectionnées

# 2. Filtrer les variables qualitatives avec une F-valeur supérieure à 1000
qualitative_vars = [var for var, results in anova_results.items() if results['F_statistic'] > 2000]

# Appliquer OrdinalEncoder sur toutes les variables qualitatives sélectionnées d'un coup
ordinal_encoder = OrdinalEncoder()
df1[qualitative_vars] = ordinal_encoder.fit_transform(df1[qualitative_vars])

# Vérification
print(df1[qualitative_vars].head())

variables_explicatives = quantitative_vars + qualitative_vars


   Méthode_application_DPE  Qualité_isolation_menuiseries  \
1                      1.0                            2.0   
2                      1.0                            2.0   
3                      1.0                            2.0   
4                      1.0                            0.0   
5                      1.0                            2.0   

   Qualité_isolation_plancher_bas  Type_énergie_n°1  \
1                             3.0              10.0   
2                             3.0               7.0   
3                             3.0               7.0   
4                             3.0              10.0   
5                             3.0              10.0   

   Qualité_isolation_enveloppe  Type_installation_chauffage  Type_bâtiment  \
1                          1.0                          1.0            0.0   
2                          0.0                          1.0            0.0   
3                          3.0                          1.0         

In [30]:
#Alorithme stepwise pour sélectionner les variables explicatives
def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, verbose=True):
    """
    Algorithme stepwise pour sélectionner les variables en utilisant le critère AIC.

    :param X: DataFrame des variables explicatives.
    :param y: Série ou tableau de la variable cible.
    :param initial_list: Liste des variables initiales (vide par défaut).
    :param threshold_in: Seuil pour ajouter une variable au modèle.
    :param threshold_out: Seuil pour retirer une variable du modèle.
    :param verbose: Affiche les informations à chaque étape si True.
    :return: Liste des variables sélectionnées.
    """
    included = list(initial_list)

    while True:
        changed = False
        # Test des ajouts de nouvelles variables
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]

        # Ajout de la variable avec la p-valeur la plus basse
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Add  {best_feature} with p-value {best_pval}')

        # Test de suppression des variables
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        pvalues = model.pvalues.iloc[1:]  # On exclut l'intercept
        worst_pval = pvalues.max()  # La p-valeur la plus haute
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Remove {worst_feature} with p-value {worst_pval}')

        if not changed:
            break

    return included

# Utilisation de la fonction stepwise_selection

X = df1[variables_explicatives]
X = X.apply(pd.to_numeric, errors='coerce')
y = df1['Conso_5_usages_é_finale']
selected_features = stepwise_selection(X, y)

print("Variables sélectionnées :", selected_features)


Add  Type_bâtiment with p-value 0.0
Add  Etiquette_DPE with p-value 0.0
Add  Surface_habitable_logement with p-value 0.0
Add  Etiquette_GES with p-value 0.0
Add  Ubat_W/m²_K with p-value 0.0
Add  Type_installation_chauffage with p-value 0.0
Add  Qualité_isolation_murs with p-value 0.0
Add  Type_énergie_n°1 with p-value 1.5203536691737305e-233
Add  Qualité_isolation_plancher_bas with p-value 8.798393091406106e-67
Add  Qualité_isolation_enveloppe with p-value 3.569885057147272e-74
Add  Méthode_application_DPE with p-value 1.4599409398257546e-41
Add  Qualité_isolation_menuiseries with p-value 6.011565708456386e-27
Variables sélectionnées : ['Type_bâtiment', 'Etiquette_DPE', 'Surface_habitable_logement', 'Etiquette_GES', 'Ubat_W/m²_K', 'Type_installation_chauffage', 'Qualité_isolation_murs', 'Type_énergie_n°1', 'Qualité_isolation_plancher_bas', 'Qualité_isolation_enveloppe', 'Méthode_application_DPE', 'Qualité_isolation_menuiseries']


### Modèles Régression 

In [31]:
#Variables explicatives à stocker 
selected_features = ['Type_bâtiment', 'Qualité_isolation_enveloppe', 'Etiquette_GES', 'Surface_habitable_logement', 'Etiquette_DPE', 'Type_installation_chauffage', 'Ubat_W/m²_K', 'Qualité_isolation_murs', 'Type_énergie_n°1', 'Qualité_isolation_plancher_bas', 'Méthode_application_DPE', 'Qualité_isolation_menuiseries']

#### Random Forest

In [33]:
# Sélection des variables quantitatives et qualitatives
quantitative_vars = [col for col in selected_features if col in df.select_dtypes(include=[np.number]).columns]
qualitative_vars = [col for col in selected_features if col in df.select_dtypes(include=['object', 'category']).columns]

# Séparation des données en ensembles d'entraînement et de test
X = df[selected_features]
y = df['Conso_5_usages_é_finale']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Définition du préprocesseur pour StandardScaler et OrdinalEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), quantitative_vars),
        ('cat', OrdinalEncoder(dtype=int), qualitative_vars)  # Encodage ordinal pour les qualitatives en entier
    ]
)

# Création du pipeline avec le préprocesseur et le modèle de régression RandomForest
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())

])

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Calcul des métriques
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichage des métriques
print("RMSE :", rmse)
print("MAE :", mae)
print("R² :", r2)


RMSE : 1089.3881426132027
MAE : 634.8763185545859
R² : 0.9582030885288537


In [ ]:
# Exporter le modèle dans un fichier avec pickle
with open('rdforest.pkl', 'wb') as file:
    pickle.dump(model, file)

#### XGBoost

In [34]:
# Sélection des variables quantitatives et qualitatives
quantitative_vars = [col for col in selected_features if col in df1.select_dtypes(include=[np.number]).columns]
qualitative_vars = [col for col in selected_features if col in df1.select_dtypes(include=['object', 'category']).columns]

X = df1[selected_features]
Y = df1['Conso_5_usages_é_finale']
# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

# Définition du préprocesseur pour StandardScaler et OrdinalEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), quantitative_vars),
        ('cat', OrdinalEncoder(dtype=int), qualitative_vars)  # Encodage ordinal en entiers pour les qualitatives
    ]
)

# Création du pipeline avec le préprocesseur et XGBRegressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

model.fit(X_train, y_train)
# Prédictions
y_pred = model.predict(X_test)

# Entraîner le modèle
# Calcul des métriques pour l'ensemble de test
rmse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Affichage des métriques
print("RMSE :", rmse)
print("MAE :", mae)
print("R2 :", r2)


RMSE : 1288393.2283536538
MAE : 759.8199597510387
R2 : 0.9546238821545451


In [ ]:
# Exporter le modèle dans un fichier avec pickle
with open('xgboost.pkl', 'wb') as file:
    pickle.dump(model, file)